In [ ]:
!pip install mumin[all]==1.6.2 torchmetrics==0.7.2 --quiet
!pip install dgl-cu111==0.7.2 -f https://data.dgl.ai/wheels/repo.html --quiet
!pip install pickle5

     |████████████████████████████████| 397 kB 11.1 MB/s 
     |████████████████████████████████| 211 kB 60.3 MB/s 
     |████████████████████████████████| 62 kB 1.8 MB/s 
     |████████████████████████████████| 6.2 MB 28.4 MB/s 
     |████████████████████████████████| 4.7 MB 75.4 MB/s 
     |████████████████████████████████| 281 kB 95.9 MB/s 
     |████████████████████████████████| 7.4 MB 50.9 MB/s 
     |████████████████████████████████| 81 kB 3.2 MB/s 
     |████████████████████████████████| 93 kB 2.5 MB/s 
     |████████████████████████████████| 6.6 MB 53.9 MB/s 
     |████████████████████████████████| 101 kB 9.5 MB/s 
     |████████████████████████████████| 165.0 MB 35 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 256 kB 7.3 MB/s 


In [ ]:
from mumin import MuminDataset
from google.colab import drive
from pathlib import Path
import shutil
import re
import pickle5 as pickle
import pandas as pd
import numpy as np

In [ ]:
drive_dir = Path('drive')
drive.mount(str(drive_dir.resolve()), force_remount=True)
drive_content_dir = [child for child in drive_dir.iterdir() 
                     if re.search(r'My ?Drive', str(child.stem)) is not None][0]
shutil.copy(drive_content_dir / 'mumin-small.zip', 'mumin-small.zip')

Mounted at /content/drive


'mumin-small.zip'

In [ ]:
with open ('/content/drive/MyDrive/mumin-small/emb/multilingual_emb_sml_tweet.pkl', 'rb') as f:
    file1 = pd.DataFrame(pickle.load(f))
with open ('/content/drive/MyDrive/mumin-small/emb/text_emb_sml_reply.pkl', 'rb') as f:
    file2 = pd.DataFrame(pickle.load(f))
with open ('/content/drive/MyDrive/mumin-small/emb/multilingual_emb_sml_reply.pkl', 'rb') as f:
    file3 = pd.DataFrame(pickle.load(f))
with open ('/content/drive/MyDrive/mumin-small/emb/text_emb_sml_tweet.pkl', 'rb') as f:
    file4 = pd.DataFrame(pickle.load(f))


In [ ]:
drive_dir = Path('drive')
drive.mount(str(drive_dir.resolve()), force_remount=True)
drive_content_dir = [child for child in drive_dir.iterdir() 
                     if re.search(r'My ?Drive', str(child.stem)) is not None][0]
shutil.copy(drive_content_dir / 'mumin-small.zip', 'mumin-small.zip')

Mounted at /content/drive


'mumin-small.zip'

In [ ]:
dataset = MuminDataset('mumin-small.zip')
dataset

MuminDataset(size=small, compiled=False)

In [ ]:
dataset.compile()

INFO:mumin.dataset:Loading dataset


MuminDataset(num_nodes=392,419, num_relations=483,029, size='small', compiled=True)

In [ ]:
dataset.nodes['tweet']['mbert_emb'] = file1['mbert_mbert_auto_4']
dataset.nodes['tweet']['text_emb'] = file4['tweet_text_auto_64']

dataset.nodes['reply']['mbert_emb'] = file3['mbert_auto_16']
dataset.nodes['reply']['text_emb'] = file2['auto_32']

In [ ]:
with open("/content/drive/MyDrive/mumin-small/emb/tweet_emb.pickle", "wb") as fp:   #Pickling
    pickle.dump(dataset.nodes['tweet'], fp)  
with open("/content/drive/MyDrive/mumin-small/emb/reply_emb.pickle", "wb") as fp:   #Pickling
    pickle.dump(dataset.nodes['reply'], fp)  


In [ ]:
drive_dir = Path('drive')
drive.mount(str(drive_dir.resolve()), force_remount=True)

Mounted at /content/drive


In [ ]:
drive_content_dir = [child for child in drive_dir.iterdir() 
                     if re.search(r'My ?Drive', str(child.stem)) is not None][0]
shutil.copy('mumin-small.zip', drive_content_dir / 'mumin-small_reduced.zip')

PosixPath('drive/MyDrive/mumin-small_reduced.zip')

In [ ]:
drive_dir = Path('drive')
drive_content_dir = [child for child in drive_dir.iterdir() 
                     if re.search(r'My ?Drive', str(child.stem)) is not None][0]
drive.mount(str(drive_dir.resolve()), force_remount=True)
shutil.copy('mumin-small.zip', drive_content_dir / 'mumin-small_reduced.zip')

Mounted at /content/drive


PosixPath('drive/MyDrive/mumin-small_reduced.zip')

In [ ]:
from PIL import Image
import itertools as it

<center><img src="https://filedn.com/lRBwPhPxgV74tO0rDoe8SpH/metagraph.png" alt="meta graph of the MuMiN dataset" width="60%"/></center>

In [ ]:
list(dataset.nodes.keys())

['claim', 'tweet', 'user', 'image', 'article', 'hashtag', 'reply']

## 5. Building a graph misinformation classifier

In [ ]:
from mumin import save_dgl_graph, load_dgl_graph
import dgl
import dgl.nn.pytorch as dglnn
import dgl.dataloading as D
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch
import torchmetrics as tm
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import defaultdict

In [ ]:
if 'dgl_graph' not in globals():
    dgl_graph = dataset.to_dgl()
dgl_graph

Graph(num_nodes={'article': 1497, 'claim': 2127, 'hashtag': 28820, 'image': 1036, 'reply': 186602, 'tweet': 4178, 'user': 158995},
      num_edges={('article', 'has_article_inv', 'tweet'): 1969, ('claim', 'discusses_inv', 'tweet'): 4890, ('hashtag', 'has_hashtag_inv', 'tweet'): 2343, ('hashtag', 'has_hashtag_inv', 'user'): 52420, ('image', 'has_image_inv', 'tweet'): 1045, ('reply', 'posted_inv', 'user'): 186602, ('reply', 'quote_of', 'tweet'): 93926, ('reply', 'reply_to', 'tweet'): 84271, ('tweet', 'discusses', 'claim'): 4890, ('tweet', 'has_article', 'article'): 1969, ('tweet', 'has_hashtag', 'hashtag'): 2343, ('tweet', 'has_image', 'image'): 1045, ('tweet', 'mentions', 'user'): 1134, ('tweet', 'posted_inv', 'user'): 4178, ('tweet', 'quote_of_inv', 'reply'): 93926, ('tweet', 'reply_to_inv', 'reply'): 84271, ('tweet', 'retweeted_inv', 'user'): 13710, ('user', 'follows', 'user'): 19886, ('user', 'follows_inv', 'user'): 19886, ('user', 'has_hashtag', 'hashtag'): 52420, ('user', 'mentions

In [ ]:
print('Node types in the DGL graph:')
print(dgl_graph.ntypes)
print('\nRelation types in the DGL graph:')
dgl_graph.canonical_etypes

Node types in the DGL graph:
['article', 'claim', 'hashtag', 'image', 'reply', 'tweet', 'user']

Relation types in the DGL graph:


[('article', 'has_article_inv', 'tweet'),
 ('claim', 'discusses_inv', 'tweet'),
 ('hashtag', 'has_hashtag_inv', 'tweet'),
 ('hashtag', 'has_hashtag_inv', 'user'),
 ('image', 'has_image_inv', 'tweet'),
 ('reply', 'posted_inv', 'user'),
 ('reply', 'quote_of', 'tweet'),
 ('reply', 'reply_to', 'tweet'),
 ('tweet', 'discusses', 'claim'),
 ('tweet', 'has_article', 'article'),
 ('tweet', 'has_hashtag', 'hashtag'),
 ('tweet', 'has_image', 'image'),
 ('tweet', 'mentions', 'user'),
 ('tweet', 'posted_inv', 'user'),
 ('tweet', 'quote_of_inv', 'reply'),
 ('tweet', 'reply_to_inv', 'reply'),
 ('tweet', 'retweeted_inv', 'user'),
 ('user', 'follows', 'user'),
 ('user', 'follows_inv', 'user'),
 ('user', 'has_hashtag', 'hashtag'),
 ('user', 'mentions', 'user'),
 ('user', 'mentions_inv', 'tweet'),
 ('user', 'mentions_inv', 'user'),
 ('user', 'posted', 'reply'),
 ('user', 'posted', 'tweet'),
 ('user', 'retweeted', 'tweet')]

## 'user', 'posted', 'tweet'

In [ ]:
rel = ('user', 'posted', 'tweet')
subgraph = dgl.edge_type_subgraph(dgl_graph, etypes=[rel]).to('cuda')
subgraph
train_mask = subgraph.nodes['tweet'].data['train_mask']
val_mask = subgraph.nodes['tweet'].data['val_mask']
test_mask = subgraph.nodes['tweet'].data['test_mask']

class SAGEClassifier(nn.Module):
    def __init__(self, hidden_dim: int = 500):
        super().__init__()
        feats1 = subgraph.nodes[rel[0]].data['feat'].shape[-1]
        feats2 = subgraph.nodes[rel[2]].data['feat'].shape[-1]
        self.conv = dglnn.SAGEConv(in_feats=(feats1, feats2), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU())
        self.clf = nn.Sequential(
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )


    def forward(self, graph, x):
        x = self.conv(graph, (x['p1'], x['p2']))
        x = self.clf(x)
        return x

gnn = SAGEClassifier().cuda()
gnn

def forward_pass() -> dict:
    '''A forward pass of the graph neural network.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Set the GNN to training mode
    gnn.train()

    # Get the input features and the output labels
    input_feats = dict(
        p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
        p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
    )
    output_labels = subgraph.nodes['tweet'].data['label'].cuda()

    # Forward propagation
    logits = gnn(subgraph, input_feats).squeeze()

    # Compute loss
    loss = F.binary_cross_entropy_with_logits(
        input=logits[train_mask],
        target=output_labels.float()[train_mask]
    )

    # Compute training metrics
    scores = scorer(logits[train_mask].ge(0), output_labels[train_mask])
    misinformation_f1 = scores[0]
    factual_f1 = scores[1]

    return dict(loss=loss, 
                misinformation_f1=misinformation_f1, 
                factual_f1=factual_f1)

def evaluate(split: str) -> dict:
    '''Evaluate the graph neural network.

    Args:
        split (str):
            The split to evaluate the GNN on. Can be 'val' or 'test'.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Get the correct mask, depending on the value of `split`
    mask = val_mask if split == 'val' else test_mask

    gnn.eval()
    with torch.no_grad():

        # Get the input features and the output labels
        input_feats = dict(
            p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
            p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
        )
        output_labels = subgraph.nodes['tweet'].data['label'].cuda()

        # Forward propagation
        logits = gnn(subgraph, input_feats).squeeze()

        # Compute validation loss
        val_loss = F.binary_cross_entropy_with_logits(
            input=logits[mask],
            target=output_labels.float()[mask]
        ).cpu().item()

        # Compute validation metrics
        scores = scorer(logits[mask].ge(0), output_labels[mask])
        val_misinformation_f1 = scores[0].cpu().item()
        val_factual_f1 = scores[1].cpu().item()

    return dict(loss=val_loss, 
                misinformation_f1=val_misinformation_f1, 
                factual_f1=val_factual_f1)

# Initialise optimiser
opt = optim.AdamW(gnn.parameters(), lr=3e-4)

# Initialise scorer
scorer = tm.classification.f_beta.F1Score(num_classes=2, average='none').cuda()

# Initialise dictionary containing validation scores
val_scores = defaultdict(list)

# Initialise progress bar
epoch_pbar = tqdm(range(1000), desc='Training')

for epoch in epoch_pbar:

    # Reset the gradients
    opt.zero_grad()

    # Forward propagation
    train_results = forward_pass()

    # Backward propagation
    train_results['loss'].backward()

    # Update gradients
    opt.step()

    # Evaluate the model
    val_results = evaluate('val')

    # Store the validation scores
    for metric in ['loss', 'misinformation_f1', 'factual_f1']:
        val_scores[metric].append(val_results[metric])

    # Update progress bar description
    if epoch % 25 == 0 and epoch > 0:
        val_loss = np.mean(val_scores['loss'])
        val_misinformation_f1 = np.mean(val_scores['misinformation_f1'])
        val_factual_f1 = np.mean(val_scores['factual_f1'])
        desc = (f'Training - '
                f'loss {train_results["loss"]:.3f} - '
                f'factual_f1 {train_results["factual_f1"]:.3f} - '
                f'misinfo_f1 {train_results["misinformation_f1"]:.3f} - '
                f'val_loss {val_loss:.3f} - '
                f'val_factual_f1 {val_factual_f1:.3f} - '
                f'val_misinfo_f1 {val_misinformation_f1:.3f}')
        epoch_pbar.set_description(desc)
        val_scores = defaultdict(list)

test_results = evaluate('test')
macro_f1 = np.mean([test_results['factual_f1'],
                    test_results['misinformation_f1']])
print()
print('*** Test results ***')
print(f'Factual F1: {test_results["factual_f1"]:.4f}')
print(f'Misinformation F1: {test_results["misinformation_f1"]:.4f}')
print(f'Macro-average F1: {macro_f1:.4f}')


Training:   2%|▏         | 23/1000 [00:01<00:47, 20.50it/s]
Training - loss 0.652 - factual_f1 0.226 - misinfo_f1 0.786 - val_loss 0.656 - val_factual_f1 0.052 - val_misinfo_f1 0.978:   2%|▏         | 23/1000 [00:01<00:47, 20.50it/s]
Training - loss 0.652 - factual_f1 0.226 - misinfo_f1 0.786 - val_loss 0.656 - val_factual_f1 0.052 - val_misinfo_f1 0.978:   3%|▎         | 26/1000 [00:01<00:47, 20.29it/s]
Training - loss 0.652 - factual_f1 0.226 - misinfo_f1 0.786 - val_loss 0.656 - val_factual_f1 0.052 - val_misinfo_f1 0.978:   3%|▎         | 29/1000 [00:01<00:47, 20.43it/s]
Training - loss 0.652 - factual_f1 0.226 - misinfo_f1 0.786 - val_loss 0.656 - val_factual_f1 0.052 - val_misinfo_f1 0.978:   3%|▎         | 32/1000 [00:01<00:47, 20.59it/s]
Training - loss 0.652 - factual_f1 0.226 - misinfo_f1 0.786 - val_loss 0.656 - val_factual_f1 0.052 - val_misinfo_f1 0.978:   4%|▎         | 35/1000 [00:01<00:46, 20.67it/s]
Training - loss 0.652 - factual_f1 0.226 - misinfo_f1 0.786 - val_los


*** Test results ***
Factual F1: 0.0909
Misinformation F1: 0.9543
Macro-average F1: 0.5226


In [ ]:
print(epoch_pbar)

Training - loss 0.141 - factual_f1 0.676 - misinfo_f1 0.972 - val_loss 1.317 - val_factual_f1 0.018 - val_misinfo_f1 0.980: 100%|██████████| 1000/1000 [00:56<00:00, 17.57it/s]


## 'reply', 'reply_to', 'tweet'

In [ ]:
rel = ('reply', 'reply_to', 'tweet')
subgraph = dgl.edge_type_subgraph(dgl_graph, etypes=[rel]).to('cuda')
subgraph

train_mask = subgraph.nodes['tweet'].data['train_mask']
val_mask = subgraph.nodes['tweet'].data['val_mask']
test_mask = subgraph.nodes['tweet'].data['test_mask']

class SAGEClassifier(nn.Module):
    def __init__(self, hidden_dim: int = 500):
        super().__init__()
        feats1 = subgraph.nodes[rel[0]].data['feat'].shape[-1]
        feats2 = subgraph.nodes[rel[2]].data['feat'].shape[-1]
        self.conv = dglnn.SAGEConv(in_feats=(feats1, feats2), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU())
        self.clf = nn.Sequential(
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )


    def forward(self, graph, x):
        x = self.conv(graph, (x['p1'], x['p2']))
        x = self.clf(x)
        return x

gnn = SAGEClassifier().cuda()
gnn

def forward_pass() -> dict:
    '''A forward pass of the graph neural network.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Set the GNN to training mode
    gnn.train()

    # Get the input features and the output labels
    input_feats = dict(
        p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
        p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
    )
    output_labels = subgraph.nodes['tweet'].data['label'].cuda()

    # Forward propagation
    logits = gnn(subgraph, input_feats).squeeze()

    # Compute loss
    loss = F.binary_cross_entropy_with_logits(
        input=logits[train_mask],
        target=output_labels.float()[train_mask]
    )

    # Compute training metrics
    scores = scorer(logits[train_mask].ge(0), output_labels[train_mask])
    misinformation_f1 = scores[0]
    factual_f1 = scores[1]

    return dict(loss=loss, 
                misinformation_f1=misinformation_f1, 
                factual_f1=factual_f1)

def evaluate(split: str) -> dict:
    '''Evaluate the graph neural network.

    Args:
        split (str):
            The split to evaluate the GNN on. Can be 'val' or 'test'.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Get the correct mask, depending on the value of `split`
    mask = val_mask if split == 'val' else test_mask

    gnn.eval()
    with torch.no_grad():

        # Get the input features and the output labels
        input_feats = dict(
            p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
            p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
        )
        output_labels = subgraph.nodes['tweet'].data['label'].cuda()

        # Forward propagation
        logits = gnn(subgraph, input_feats).squeeze()

        # Compute validation loss
        val_loss = F.binary_cross_entropy_with_logits(
            input=logits[mask],
            target=output_labels.float()[mask]
        ).cpu().item()

        # Compute validation metrics
        scores = scorer(logits[mask].ge(0), output_labels[mask])
        val_misinformation_f1 = scores[0].cpu().item()
        val_factual_f1 = scores[1].cpu().item()

    return dict(loss=val_loss, 
                misinformation_f1=val_misinformation_f1, 
                factual_f1=val_factual_f1)

# Initialise optimiser
opt = optim.AdamW(gnn.parameters(), lr=3e-4)

# Initialise scorer
scorer = tm.classification.f_beta.F1Score(num_classes=2, average='none').cuda()

# Initialise dictionary containing validation scores
val_scores = defaultdict(list)

# Initialise progress bar
epoch_pbar = tqdm(range(1000), desc='Training')

for epoch in epoch_pbar:

    # Reset the gradients
    opt.zero_grad()

    # Forward propagation
    train_results = forward_pass()

    # Backward propagation
    train_results['loss'].backward()

    # Update gradients
    opt.step()

    # Evaluate the model
    val_results = evaluate('val')

    # Store the validation scores
    for metric in ['loss', 'misinformation_f1', 'factual_f1']:
        val_scores[metric].append(val_results[metric])

    # Update progress bar description
    if epoch % 25 == 0 and epoch > 0:
        val_loss = np.mean(val_scores['loss'])
        val_misinformation_f1 = np.mean(val_scores['misinformation_f1'])
        val_factual_f1 = np.mean(val_scores['factual_f1'])
        desc = (f'Training - '
                f'loss {train_results["loss"]:.3f} - '
                f'factual_f1 {train_results["factual_f1"]:.3f} - '
                f'misinfo_f1 {train_results["misinformation_f1"]:.3f} - '
                f'val_loss {val_loss:.3f} - '
                f'val_factual_f1 {val_factual_f1:.3f} - '
                f'val_misinfo_f1 {val_misinformation_f1:.3f}')
        epoch_pbar.set_description(desc)
        val_scores = defaultdict(list)

test_results = evaluate('test')
macro_f1 = np.mean([test_results['factual_f1'],
                    test_results['misinformation_f1']])
print()
print('*** Test results ***')
print(f'Factual F1: {test_results["factual_f1"]:.4f}')
print(f'Misinformation F1: {test_results["misinformation_f1"]:.4f}')
print(f'Macro-average F1: {macro_f1:.4f}')


Training:   2%|▎         | 25/1000 [00:17<11:24,  1.42it/s]
Training - loss 0.678 - factual_f1 0.191 - misinfo_f1 0.702 - val_loss 0.689 - val_factual_f1 0.025 - val_misinfo_f1 0.536:   2%|▎         | 25/1000 [00:18<11:24,  1.42it/s]
Training - loss 0.678 - factual_f1 0.191 - misinfo_f1 0.702 - val_loss 0.689 - val_factual_f1 0.025 - val_misinfo_f1 0.536:   3%|▎         | 26/1000 [00:18<11:26,  1.42it/s]
Training - loss 0.678 - factual_f1 0.191 - misinfo_f1 0.702 - val_loss 0.689 - val_factual_f1 0.025 - val_misinfo_f1 0.536:   3%|▎         | 27/1000 [00:19<11:23,  1.42it/s]
Training - loss 0.678 - factual_f1 0.191 - misinfo_f1 0.702 - val_loss 0.689 - val_factual_f1 0.025 - val_misinfo_f1 0.536:   3%|▎         | 28/1000 [00:20<11:23,  1.42it/s]
Training - loss 0.678 - factual_f1 0.191 - misinfo_f1 0.702 - val_loss 0.689 - val_factual_f1 0.025 - val_misinfo_f1 0.536:   3%|▎         | 29/1000 [00:20<11:24,  1.42it/s]
Training - loss 0.678 - factual_f1 0.191 - misinfo_f1 0.702 - val_los

*** Test results ***
Factual F1: 0.0476
Misinformation F1: 0.9544
Macro-average F1: 0.5010


## 'claim', 'discusses_inv', 'tweet'


In [ ]:
rel = ('claim', 'discusses_inv', 'tweet')
subgraph = dgl.edge_type_subgraph(dgl_graph, etypes=[rel]).to('cuda')
subgraph

train_mask = subgraph.nodes['tweet'].data['train_mask']
val_mask = subgraph.nodes['tweet'].data['val_mask']
test_mask = subgraph.nodes['tweet'].data['test_mask']

class SAGEClassifier(nn.Module):
    def __init__(self, hidden_dim: int = 500):
        super().__init__()
        feats1 = subgraph.nodes[rel[0]].data['feat'].shape[-1]
        feats2 = subgraph.nodes[rel[2]].data['feat'].shape[-1]
        self.conv = dglnn.SAGEConv(in_feats=(feats1, feats2), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU())
        self.clf = nn.Sequential(
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )


    def forward(self, graph, x):
        x = self.conv(graph, (x['p1'], x['p2']))
        x = self.clf(x)
        return x

gnn = SAGEClassifier().cuda()
gnn

def forward_pass() -> dict:
    '''A forward pass of the graph neural network.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Set the GNN to training mode
    gnn.train()

    # Get the input features and the output labels
    input_feats = dict(
        p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
        p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
    )
    output_labels = subgraph.nodes['tweet'].data['label'].cuda()

    # Forward propagation
    logits = gnn(subgraph, input_feats).squeeze()

    # Compute loss
    loss = F.binary_cross_entropy_with_logits(
        input=logits[train_mask],
        target=output_labels.float()[train_mask]
    )

    # Compute training metrics
    scores = scorer(logits[train_mask].ge(0), output_labels[train_mask])
    misinformation_f1 = scores[0]
    factual_f1 = scores[1]

    return dict(loss=loss, 
                misinformation_f1=misinformation_f1, 
                factual_f1=factual_f1)

def evaluate(split: str) -> dict:
    '''Evaluate the graph neural network.

    Args:
        split (str):
            The split to evaluate the GNN on. Can be 'val' or 'test'.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Get the correct mask, depending on the value of `split`
    mask = val_mask if split == 'val' else test_mask

    gnn.eval()
    with torch.no_grad():

        # Get the input features and the output labels
        input_feats = dict(
            p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
            p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
        )
        output_labels = subgraph.nodes['tweet'].data['label'].cuda()

        # Forward propagation
        logits = gnn(subgraph, input_feats).squeeze()

        # Compute validation loss
        val_loss = F.binary_cross_entropy_with_logits(
            input=logits[mask],
            target=output_labels.float()[mask]
        ).cpu().item()

        # Compute validation metrics
        scores = scorer(logits[mask].ge(0), output_labels[mask])
        val_misinformation_f1 = scores[0].cpu().item()
        val_factual_f1 = scores[1].cpu().item()

    return dict(loss=val_loss, 
                misinformation_f1=val_misinformation_f1, 
                factual_f1=val_factual_f1)

# Initialise optimiser
opt = optim.AdamW(gnn.parameters(), lr=3e-4)

# Initialise scorer
scorer = tm.classification.f_beta.F1Score(num_classes=2, average='none').cuda()

# Initialise dictionary containing validation scores
val_scores = defaultdict(list)

# Initialise progress bar
epoch_pbar = tqdm(range(1000), desc='Training')

for epoch in epoch_pbar:

    # Reset the gradients
    opt.zero_grad()

    # Forward propagation
    train_results = forward_pass()

    # Backward propagation
    train_results['loss'].backward()

    # Update gradients
    opt.step()

    # Evaluate the model
    val_results = evaluate('val')

    # Store the validation scores
    for metric in ['loss', 'misinformation_f1', 'factual_f1']:
        val_scores[metric].append(val_results[metric])

    # Update progress bar description
    if epoch % 25 == 0 and epoch > 0:
        val_loss = np.mean(val_scores['loss'])
        val_misinformation_f1 = np.mean(val_scores['misinformation_f1'])
        val_factual_f1 = np.mean(val_scores['factual_f1'])
        desc = (f'Training - '
                f'loss {train_results["loss"]:.3f} - '
                f'factual_f1 {train_results["factual_f1"]:.3f} - '
                f'misinfo_f1 {train_results["misinformation_f1"]:.3f} - '
                f'val_loss {val_loss:.3f} - '
                f'val_factual_f1 {val_factual_f1:.3f} - '
                f'val_misinfo_f1 {val_misinformation_f1:.3f}')
        epoch_pbar.set_description(desc)
        val_scores = defaultdict(list)

test_results = evaluate('test')
macro_f1 = np.mean([test_results['factual_f1'],
                    test_results['misinformation_f1']])
print()
print('*** Test results ***')
print(f'Factual F1: {test_results["factual_f1"]:.4f}')
print(f'Misinformation F1: {test_results["misinformation_f1"]:.4f}')
print(f'Macro-average F1: {macro_f1:.4f}')

Training - loss 0.000 - factual_f1 1.000 - misinfo_f1 1.000 - val_loss 22.023 - val_factual_f1 0.022 - val_misinfo_f1 0.332: 100%|██████████| 1000/1000 [01:22<00:00, 12.17it/s]


*** Test results ***
Factual F1: 0.1097
Misinformation F1: 0.5475
Macro-average F1: 0.3286


## 'reply', 'quote_of', 'tweet'

In [ ]:
rel = ('reply', 'quote_of', 'tweet')
subgraph = dgl.edge_type_subgraph(dgl_graph, etypes=[rel]).to('cuda')
subgraph

train_mask = subgraph.nodes['tweet'].data['train_mask']
val_mask = subgraph.nodes['tweet'].data['val_mask']
test_mask = subgraph.nodes['tweet'].data['test_mask']

class SAGEClassifier(nn.Module):
    def __init__(self, hidden_dim: int = 500):
        super().__init__()
        feats1 = subgraph.nodes[rel[0]].data['feat'].shape[-1]
        feats2 = subgraph.nodes[rel[2]].data['feat'].shape[-1]
        self.conv = dglnn.SAGEConv(in_feats=(feats1, feats2), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU())
        self.clf = nn.Sequential(
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )


    def forward(self, graph, x):
        x = self.conv(graph, (x['p1'], x['p2']))
        x = self.clf(x)
        return x

gnn = SAGEClassifier().cuda()
gnn

def forward_pass() -> dict:
    '''A forward pass of the graph neural network.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Set the GNN to training mode
    gnn.train()

    # Get the input features and the output labels
    input_feats = dict(
        p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
        p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
    )
    output_labels = subgraph.nodes['tweet'].data['label'].cuda()

    # Forward propagation
    logits = gnn(subgraph, input_feats).squeeze()

    # Compute loss
    loss = F.binary_cross_entropy_with_logits(
        input=logits[train_mask],
        target=output_labels.float()[train_mask]
    )

    # Compute training metrics
    scores = scorer(logits[train_mask].ge(0), output_labels[train_mask])
    misinformation_f1 = scores[0]
    factual_f1 = scores[1]

    return dict(loss=loss, 
                misinformation_f1=misinformation_f1, 
                factual_f1=factual_f1)

def evaluate(split: str) -> dict:
    '''Evaluate the graph neural network.

    Args:
        split (str):
            The split to evaluate the GNN on. Can be 'val' or 'test'.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Get the correct mask, depending on the value of `split`
    mask = val_mask if split == 'val' else test_mask

    gnn.eval()
    with torch.no_grad():

        # Get the input features and the output labels
        input_feats = dict(
            p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
            p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
        )
        output_labels = subgraph.nodes['tweet'].data['label'].cuda()

        # Forward propagation
        logits = gnn(subgraph, input_feats).squeeze()

        # Compute validation loss
        val_loss = F.binary_cross_entropy_with_logits(
            input=logits[mask],
            target=output_labels.float()[mask]
        ).cpu().item()

        # Compute validation metrics
        scores = scorer(logits[mask].ge(0), output_labels[mask])
        val_misinformation_f1 = scores[0].cpu().item()
        val_factual_f1 = scores[1].cpu().item()

    return dict(loss=val_loss, 
                misinformation_f1=val_misinformation_f1, 
                factual_f1=val_factual_f1)

# Initialise optimiser
opt = optim.AdamW(gnn.parameters(), lr=3e-4)

# Initialise scorer
scorer = tm.classification.f_beta.F1Score(num_classes=2, average='none').cuda()

# Initialise dictionary containing validation scores
val_scores = defaultdict(list)

# Initialise progress bar
epoch_pbar = tqdm(range(1000), desc='Training')

for epoch in epoch_pbar:

    # Reset the gradients
    opt.zero_grad()

    # Forward propagation
    train_results = forward_pass()

    # Backward propagation
    train_results['loss'].backward()

    # Update gradients
    opt.step()

    # Evaluate the model
    val_results = evaluate('val')

    # Store the validation scores
    for metric in ['loss', 'misinformation_f1', 'factual_f1']:
        val_scores[metric].append(val_results[metric])

    # Update progress bar description
    if epoch % 25 == 0 and epoch > 0:
        val_loss = np.mean(val_scores['loss'])
        val_misinformation_f1 = np.mean(val_scores['misinformation_f1'])
        val_factual_f1 = np.mean(val_scores['factual_f1'])
        desc = (f'Training - '
                f'loss {train_results["loss"]:.3f} - '
                f'factual_f1 {train_results["factual_f1"]:.3f} - '
                f'misinfo_f1 {train_results["misinformation_f1"]:.3f} - '
                f'val_loss {val_loss:.3f} - '
                f'val_factual_f1 {val_factual_f1:.3f} - '
                f'val_misinfo_f1 {val_misinformation_f1:.3f}')
        epoch_pbar.set_description(desc)
        val_scores = defaultdict(list)

test_results = evaluate('test')
macro_f1 = np.mean([test_results['factual_f1'],
                    test_results['misinformation_f1']])
print()
print('*** Test results ***')
print(f'Factual F1: {test_results["factual_f1"]:.4f}')
print(f'Misinformation F1: {test_results["misinformation_f1"]:.4f}')
print(f'Macro-average F1: {macro_f1:.4f}')

Training - loss 0.133 - factual_f1 0.712 - misinfo_f1 0.976 - val_loss 0.522 - val_factual_f1 0.165 - val_misinfo_f1 0.971: 100%|██████████| 1000/1000 [12:20<00:00,  1.35it/s]



*** Test results ***
Factual F1: 0.1967
Misinformation F1: 0.9430
Macro-average F1: 0.5698


## 'article', 'has_article_inv', 'tweet'

In [ ]:
rel = ('article', 'has_article_inv', 'tweet')
subgraph = dgl.edge_type_subgraph(dgl_graph, etypes=[rel]).to('cuda')
subgraph

train_mask = subgraph.nodes['tweet'].data['train_mask']
val_mask = subgraph.nodes['tweet'].data['val_mask']
test_mask = subgraph.nodes['tweet'].data['test_mask']

class SAGEClassifier(nn.Module):
    def __init__(self, hidden_dim: int = 500):
        super().__init__()
        feats1 = subgraph.nodes[rel[0]].data['feat'].shape[-1]
        feats2 = subgraph.nodes[rel[2]].data['feat'].shape[-1]
        self.conv = dglnn.SAGEConv(in_feats=(feats1, feats2), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU())
        self.clf = nn.Sequential(
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )


    def forward(self, graph, x):
        x = self.conv(graph, (x['p1'], x['p2']))
        x = self.clf(x)
        return x

gnn = SAGEClassifier().cuda()
gnn

def forward_pass() -> dict:
    '''A forward pass of the graph neural network.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Set the GNN to training mode
    gnn.train()

    # Get the input features and the output labels
    input_feats = dict(
        p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
        p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
    )
    output_labels = subgraph.nodes['tweet'].data['label'].cuda()

    # Forward propagation
    logits = gnn(subgraph, input_feats).squeeze()

    # Compute loss
    loss = F.binary_cross_entropy_with_logits(
        input=logits[train_mask],
        target=output_labels.float()[train_mask]
    )

    # Compute training metrics
    scores = scorer(logits[train_mask].ge(0), output_labels[train_mask])
    misinformation_f1 = scores[0]
    factual_f1 = scores[1]

    return dict(loss=loss, 
                misinformation_f1=misinformation_f1, 
                factual_f1=factual_f1)

def evaluate(split: str) -> dict:
    '''Evaluate the graph neural network.

    Args:
        split (str):
            The split to evaluate the GNN on. Can be 'val' or 'test'.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Get the correct mask, depending on the value of `split`
    mask = val_mask if split == 'val' else test_mask

    gnn.eval()
    with torch.no_grad():

        # Get the input features and the output labels
        input_feats = dict(
            p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
            p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
        )
        output_labels = subgraph.nodes['tweet'].data['label'].cuda()

        # Forward propagation
        logits = gnn(subgraph, input_feats).squeeze()

        # Compute validation loss
        val_loss = F.binary_cross_entropy_with_logits(
            input=logits[mask],
            target=output_labels.float()[mask]
        ).cpu().item()

        # Compute validation metrics
        scores = scorer(logits[mask].ge(0), output_labels[mask])
        val_misinformation_f1 = scores[0].cpu().item()
        val_factual_f1 = scores[1].cpu().item()

    return dict(loss=val_loss, 
                misinformation_f1=val_misinformation_f1, 
                factual_f1=val_factual_f1)

# Initialise optimiser
opt = optim.AdamW(gnn.parameters(), lr=3e-4)

# Initialise scorer
scorer = tm.classification.f_beta.F1Score(num_classes=2, average='none').cuda()

# Initialise dictionary containing validation scores
val_scores = defaultdict(list)

# Initialise progress bar
epoch_pbar = tqdm(range(1000), desc='Training')

for epoch in epoch_pbar:

    # Reset the gradients
    opt.zero_grad()

    # Forward propagation
    train_results = forward_pass()

    # Backward propagation
    train_results['loss'].backward()

    # Update gradients
    opt.step()

    # Evaluate the model
    val_results = evaluate('val')

    # Store the validation scores
    for metric in ['loss', 'misinformation_f1', 'factual_f1']:
        val_scores[metric].append(val_results[metric])

    # Update progress bar description
    if epoch % 25 == 0 and epoch > 0:
        val_loss = np.mean(val_scores['loss'])
        val_misinformation_f1 = np.mean(val_scores['misinformation_f1'])
        val_factual_f1 = np.mean(val_scores['factual_f1'])
        desc = (f'Training - '
                f'loss {train_results["loss"]:.3f} - '
                f'factual_f1 {train_results["factual_f1"]:.3f} - '
                f'misinfo_f1 {train_results["misinformation_f1"]:.3f} - '
                f'val_loss {val_loss:.3f} - '
                f'val_factual_f1 {val_factual_f1:.3f} - '
                f'val_misinfo_f1 {val_misinformation_f1:.3f}')
        epoch_pbar.set_description(desc)
        val_scores = defaultdict(list)

test_results = evaluate('test')
macro_f1 = np.mean([test_results['factual_f1'],
                    test_results['misinformation_f1']])
print()
print('*** Test results ***')
print(f'Factual F1: {test_results["factual_f1"]:.4f}')
print(f'Misinformation F1: {test_results["misinformation_f1"]:.4f}')
print(f'Macro-average F1: {macro_f1:.4f}')

Training - loss 0.168 - factual_f1 0.647 - misinfo_f1 0.971 - val_loss 0.205 - val_factual_f1 0.053 - val_misinfo_f1 0.976: 100%|██████████| 1000/1000 [01:18<00:00, 12.81it/s]


*** Test results ***
Factual F1: 0.1875
Misinformation F1: 0.9707
Macro-average F1: 0.5791


## 'hashtag', 'has_hashtag_inv', 'tweet'

In [ ]:
rel = ('hashtag', 'has_hashtag_inv', 'tweet')
subgraph = dgl.edge_type_subgraph(dgl_graph, etypes=[rel]).to('cuda')
subgraph

train_mask = subgraph.nodes['tweet'].data['train_mask']
val_mask = subgraph.nodes['tweet'].data['val_mask']
test_mask = subgraph.nodes['tweet'].data['test_mask']

class SAGEClassifier(nn.Module):
    def __init__(self, hidden_dim: int = 500):
        super().__init__()
        feats1 = subgraph.nodes[rel[0]].data['feat'].shape[-1]
        feats2 = subgraph.nodes[rel[2]].data['feat'].shape[-1]
        self.conv = dglnn.SAGEConv(in_feats=(feats1, feats2), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU())
        self.clf = nn.Sequential(
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )


    def forward(self, graph, x):
        x = self.conv(graph, (x['p1'], x['p2']))
        x = self.clf(x)
        return x

gnn = SAGEClassifier().cuda()
gnn

def forward_pass() -> dict:
    '''A forward pass of the graph neural network.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Set the GNN to training mode
    gnn.train()

    # Get the input features and the output labels
    input_feats = dict(
        p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
        p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
    )
    output_labels = subgraph.nodes['tweet'].data['label'].cuda()

    # Forward propagation
    logits = gnn(subgraph, input_feats).squeeze()

    # Compute loss
    loss = F.binary_cross_entropy_with_logits(
        input=logits[train_mask],
        target=output_labels.float()[train_mask]
    )

    # Compute training metrics
    scores = scorer(logits[train_mask].ge(0), output_labels[train_mask])
    misinformation_f1 = scores[0]
    factual_f1 = scores[1]

    return dict(loss=loss, 
                misinformation_f1=misinformation_f1, 
                factual_f1=factual_f1)

def evaluate(split: str) -> dict:
    '''Evaluate the graph neural network.

    Args:
        split (str):
            The split to evaluate the GNN on. Can be 'val' or 'test'.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Get the correct mask, depending on the value of `split`
    mask = val_mask if split == 'val' else test_mask

    gnn.eval()
    with torch.no_grad():

        # Get the input features and the output labels
        input_feats = dict(
            p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
            p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
        )
        output_labels = subgraph.nodes['tweet'].data['label'].cuda()

        # Forward propagation
        logits = gnn(subgraph, input_feats).squeeze()

        # Compute validation loss
        val_loss = F.binary_cross_entropy_with_logits(
            input=logits[mask],
            target=output_labels.float()[mask]
        ).cpu().item()

        # Compute validation metrics
        scores = scorer(logits[mask].ge(0), output_labels[mask])
        val_misinformation_f1 = scores[0].cpu().item()
        val_factual_f1 = scores[1].cpu().item()

    return dict(loss=val_loss, 
                misinformation_f1=val_misinformation_f1, 
                factual_f1=val_factual_f1)

# Initialise optimiser
opt = optim.AdamW(gnn.parameters(), lr=3e-4)

# Initialise scorer
scorer = tm.classification.f_beta.F1Score(num_classes=2, average='none').cuda()

# Initialise dictionary containing validation scores
val_scores = defaultdict(list)

# Initialise progress bar
epoch_pbar = tqdm(range(1000), desc='Training')

for epoch in epoch_pbar:

    # Reset the gradients
    opt.zero_grad()

    # Forward propagation
    train_results = forward_pass()

    # Backward propagation
    train_results['loss'].backward()

    # Update gradients
    opt.step()

    # Evaluate the model
    val_results = evaluate('val')

    # Store the validation scores
    for metric in ['loss', 'misinformation_f1', 'factual_f1']:
        val_scores[metric].append(val_results[metric])

    # Update progress bar description
    if epoch % 25 == 0 and epoch > 0:
        val_loss = np.mean(val_scores['loss'])
        val_misinformation_f1 = np.mean(val_scores['misinformation_f1'])
        val_factual_f1 = np.mean(val_scores['factual_f1'])
        desc = (f'Training - '
                f'loss {train_results["loss"]:.3f} - '
                f'factual_f1 {train_results["factual_f1"]:.3f} - '
                f'misinfo_f1 {train_results["misinformation_f1"]:.3f} - '
                f'val_loss {val_loss:.3f} - '
                f'val_factual_f1 {val_factual_f1:.3f} - '
                f'val_misinfo_f1 {val_misinformation_f1:.3f}')
        epoch_pbar.set_description(desc)
        val_scores = defaultdict(list)

test_results = evaluate('test')
macro_f1 = np.mean([test_results['factual_f1'],
                    test_results['misinformation_f1']])
print()
print('*** Test results ***')
print(f'Factual F1: {test_results["factual_f1"]:.4f}')
print(f'Misinformation F1: {test_results["misinformation_f1"]:.4f}')
print(f'Macro-average F1: {macro_f1:.4f}')

Training - loss 0.254 - factual_f1 0.192 - misinfo_f1 0.953 - val_loss 0.121 - val_factual_f1 0.015 - val_misinfo_f1 0.991: 100%|██████████| 1000/1000 [00:39<00:00, 25.61it/s]


*** Test results ***
Factual F1: 0.0833
Misinformation F1: 0.9754
Macro-average F1: 0.5294


## 'image', 'has_image_inv', 'tweet'

In [ ]:
rel = ('image', 'has_image_inv', 'tweet')
subgraph = dgl.edge_type_subgraph(dgl_graph, etypes=[rel]).to('cuda')
subgraph

train_mask = subgraph.nodes['tweet'].data['train_mask']
val_mask = subgraph.nodes['tweet'].data['val_mask']
test_mask = subgraph.nodes['tweet'].data['test_mask']

class SAGEClassifier(nn.Module):
    def __init__(self, hidden_dim: int = 500):
        super().__init__()
        feats1 = subgraph.nodes[rel[0]].data['feat'].shape[-1]
        feats2 = subgraph.nodes[rel[2]].data['feat'].shape[-1]
        self.conv = dglnn.SAGEConv(in_feats=(feats1, feats2), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU())
        self.clf = nn.Sequential(
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )


    def forward(self, graph, x):
        x = self.conv(graph, (x['p1'], x['p2']))
        x = self.clf(x)
        return x

gnn = SAGEClassifier().cuda()
gnn

def forward_pass() -> dict:
    '''A forward pass of the graph neural network.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Set the GNN to training mode
    gnn.train()

    # Get the input features and the output labels
    input_feats = dict(
        p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
        p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
    )
    output_labels = subgraph.nodes['tweet'].data['label'].cuda()

    # Forward propagation
    logits = gnn(subgraph, input_feats).squeeze()

    # Compute loss
    loss = F.binary_cross_entropy_with_logits(
        input=logits[train_mask],
        target=output_labels.float()[train_mask]
    )

    # Compute training metrics
    scores = scorer(logits[train_mask].ge(0), output_labels[train_mask])
    misinformation_f1 = scores[0]
    factual_f1 = scores[1]

    return dict(loss=loss, 
                misinformation_f1=misinformation_f1, 
                factual_f1=factual_f1)

def evaluate(split: str) -> dict:
    '''Evaluate the graph neural network.

    Args:
        split (str):
            The split to evaluate the GNN on. Can be 'val' or 'test'.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Get the correct mask, depending on the value of `split`
    mask = val_mask if split == 'val' else test_mask

    gnn.eval()
    with torch.no_grad():

        # Get the input features and the output labels
        input_feats = dict(
            p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
            p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
        )
        output_labels = subgraph.nodes['tweet'].data['label'].cuda()

        # Forward propagation
        logits = gnn(subgraph, input_feats).squeeze()

        # Compute validation loss
        val_loss = F.binary_cross_entropy_with_logits(
            input=logits[mask],
            target=output_labels.float()[mask]
        ).cpu().item()

        # Compute validation metrics
        scores = scorer(logits[mask].ge(0), output_labels[mask])
        val_misinformation_f1 = scores[0].cpu().item()
        val_factual_f1 = scores[1].cpu().item()

    return dict(loss=val_loss, 
                misinformation_f1=val_misinformation_f1, 
                factual_f1=val_factual_f1)

# Initialise optimiser
opt = optim.AdamW(gnn.parameters(), lr=3e-4)

# Initialise scorer
scorer = tm.classification.f_beta.F1Score(num_classes=2, average='none').cuda()

# Initialise dictionary containing validation scores
val_scores = defaultdict(list)

# Initialise progress bar
epoch_pbar = tqdm(range(1000), desc='Training')

for epoch in epoch_pbar:

    # Reset the gradients
    opt.zero_grad()

    # Forward propagation
    train_results = forward_pass()

    # Backward propagation
    train_results['loss'].backward()

    # Update gradients
    opt.step()

    # Evaluate the model
    val_results = evaluate('val')

    # Store the validation scores
    for metric in ['loss', 'misinformation_f1', 'factual_f1']:
        val_scores[metric].append(val_results[metric])

    # Update progress bar description
    if epoch % 25 == 0 and epoch > 0:
        val_loss = np.mean(val_scores['loss'])
        val_misinformation_f1 = np.mean(val_scores['misinformation_f1'])
        val_factual_f1 = np.mean(val_scores['factual_f1'])
        desc = (f'Training - '
                f'loss {train_results["loss"]:.3f} - '
                f'factual_f1 {train_results["factual_f1"]:.3f} - '
                f'misinfo_f1 {train_results["misinformation_f1"]:.3f} - '
                f'val_loss {val_loss:.3f} - '
                f'val_factual_f1 {val_factual_f1:.3f} - '
                f'val_misinfo_f1 {val_misinformation_f1:.3f}')
        epoch_pbar.set_description(desc)
        val_scores = defaultdict(list)

test_results = evaluate('test')
macro_f1 = np.mean([test_results['factual_f1'],
                    test_results['misinformation_f1']])
print()
print('*** Test results ***')
print(f'Factual F1: {test_results["factual_f1"]:.4f}')
print(f'Misinformation F1: {test_results["misinformation_f1"]:.4f}')
print(f'Macro-average F1: {macro_f1:.4f}')

Training - loss 0.193 - factual_f1 0.507 - misinfo_f1 0.965 - val_loss 9.734 - val_factual_f1 0.035 - val_misinfo_f1 0.887: 100%|██████████| 1000/1000 [00:30<00:00, 33.18it/s]


*** Test results ***
Factual F1: 0.1538
Misinformation F1: 0.8922
Macro-average F1: 0.5230


## 'user', 'mentions_inv', 'tweet'

In [ ]:
rel = ('user', 'mentions_inv', 'tweet')
subgraph = dgl.edge_type_subgraph(dgl_graph, etypes=[rel]).to('cuda')
subgraph

train_mask = subgraph.nodes['tweet'].data['train_mask']
val_mask = subgraph.nodes['tweet'].data['val_mask']
test_mask = subgraph.nodes['tweet'].data['test_mask']

class SAGEClassifier(nn.Module):
    def __init__(self, hidden_dim: int = 500):
        super().__init__()
        feats1 = subgraph.nodes[rel[0]].data['feat'].shape[-1]
        feats2 = subgraph.nodes[rel[2]].data['feat'].shape[-1]
        self.conv = dglnn.SAGEConv(in_feats=(feats1, feats2), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU())
        self.clf = nn.Sequential(
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )


    def forward(self, graph, x):
        x = self.conv(graph, (x['p1'], x['p2']))
        x = self.clf(x)
        return x

gnn = SAGEClassifier().cuda()
gnn

def forward_pass() -> dict:
    '''A forward pass of the graph neural network.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Set the GNN to training mode
    gnn.train()

    # Get the input features and the output labels
    input_feats = dict(
        p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
        p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
    )
    output_labels = subgraph.nodes['tweet'].data['label'].cuda()

    # Forward propagation
    logits = gnn(subgraph, input_feats).squeeze()

    # Compute loss
    loss = F.binary_cross_entropy_with_logits(
        input=logits[train_mask],
        target=output_labels.float()[train_mask]
    )

    # Compute training metrics
    scores = scorer(logits[train_mask].ge(0), output_labels[train_mask])
    misinformation_f1 = scores[0]
    factual_f1 = scores[1]

    return dict(loss=loss, 
                misinformation_f1=misinformation_f1, 
                factual_f1=factual_f1)

def evaluate(split: str) -> dict:
    '''Evaluate the graph neural network.

    Args:
        split (str):
            The split to evaluate the GNN on. Can be 'val' or 'test'.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Get the correct mask, depending on the value of `split`
    mask = val_mask if split == 'val' else test_mask

    gnn.eval()
    with torch.no_grad():

        # Get the input features and the output labels
        input_feats = dict(
            p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
            p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
        )
        output_labels = subgraph.nodes['tweet'].data['label'].cuda()

        # Forward propagation
        logits = gnn(subgraph, input_feats).squeeze()

        # Compute validation loss
        val_loss = F.binary_cross_entropy_with_logits(
            input=logits[mask],
            target=output_labels.float()[mask]
        ).cpu().item()

        # Compute validation metrics
        scores = scorer(logits[mask].ge(0), output_labels[mask])
        val_misinformation_f1 = scores[0].cpu().item()
        val_factual_f1 = scores[1].cpu().item()

    return dict(loss=val_loss, 
                misinformation_f1=val_misinformation_f1, 
                factual_f1=val_factual_f1)

# Initialise optimiser
opt = optim.AdamW(gnn.parameters(), lr=3e-4)

# Initialise scorer
scorer = tm.classification.f_beta.F1Score(num_classes=2, average='none').cuda()

# Initialise dictionary containing validation scores
val_scores = defaultdict(list)

# Initialise progress bar
epoch_pbar = tqdm(range(1000), desc='Training')

for epoch in epoch_pbar:

    # Reset the gradients
    opt.zero_grad()

    # Forward propagation
    train_results = forward_pass()

    # Backward propagation
    train_results['loss'].backward()

    # Update gradients
    opt.step()

    # Evaluate the model
    val_results = evaluate('val')

    # Store the validation scores
    for metric in ['loss', 'misinformation_f1', 'factual_f1']:
        val_scores[metric].append(val_results[metric])

    # Update progress bar description
    if epoch % 25 == 0 and epoch > 0:
        val_loss = np.mean(val_scores['loss'])
        val_misinformation_f1 = np.mean(val_scores['misinformation_f1'])
        val_factual_f1 = np.mean(val_scores['factual_f1'])
        desc = (f'Training - '
                f'loss {train_results["loss"]:.3f} - '
                f'factual_f1 {train_results["factual_f1"]:.3f} - '
                f'misinfo_f1 {train_results["misinformation_f1"]:.3f} - '
                f'val_loss {val_loss:.3f} - '
                f'val_factual_f1 {val_factual_f1:.3f} - '
                f'val_misinfo_f1 {val_misinformation_f1:.3f}')
        epoch_pbar.set_description(desc)
        val_scores = defaultdict(list)

test_results = evaluate('test')
macro_f1 = np.mean([test_results['factual_f1'],
                    test_results['misinformation_f1']])
print()
print('*** Test results ***')
print(f'Factual F1: {test_results["factual_f1"]:.4f}')
print(f'Misinformation F1: {test_results["misinformation_f1"]:.4f}')
print(f'Macro-average F1: {macro_f1:.4f}')

Training - loss 0.224 - factual_f1 0.356 - misinfo_f1 0.958 - val_loss 0.897 - val_factual_f1 0.075 - val_misinfo_f1 0.977: 100%|██████████| 1000/1000 [01:03<00:00, 15.82it/s]


*** Test results ***
Factual F1: 0.0000
Misinformation F1: 0.9474
Macro-average F1: 0.4737


## 'user', 'retweeted', 'tweet'

In [ ]:
rel = ('user', 'retweeted', 'tweet')
subgraph = dgl.edge_type_subgraph(dgl_graph, etypes=[rel]).to('cuda')
subgraph

train_mask = subgraph.nodes['tweet'].data['train_mask']
val_mask = subgraph.nodes['tweet'].data['val_mask']
test_mask = subgraph.nodes['tweet'].data['test_mask']

class SAGEClassifier(nn.Module):
    def __init__(self, hidden_dim: int = 500):
        super().__init__()
        feats1 = subgraph.nodes[rel[0]].data['feat'].shape[-1]
        feats2 = subgraph.nodes[rel[2]].data['feat'].shape[-1]
        self.conv = dglnn.SAGEConv(in_feats=(feats1, feats2), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU())
        self.clf = nn.Sequential(
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )


    def forward(self, graph, x):
        x = self.conv(graph, (x['p1'], x['p2']))
        x = self.clf(x)
        return x

gnn = SAGEClassifier().cuda()
gnn

def forward_pass() -> dict:
    '''A forward pass of the graph neural network.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Set the GNN to training mode
    gnn.train()

    # Get the input features and the output labels
    input_feats = dict(
        p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
        p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
    )
    output_labels = subgraph.nodes['tweet'].data['label'].cuda()

    # Forward propagation
    logits = gnn(subgraph, input_feats).squeeze()

    # Compute loss
    loss = F.binary_cross_entropy_with_logits(
        input=logits[train_mask],
        target=output_labels.float()[train_mask]
    )

    # Compute training metrics
    scores = scorer(logits[train_mask].ge(0), output_labels[train_mask])
    misinformation_f1 = scores[0]
    factual_f1 = scores[1]

    return dict(loss=loss, 
                misinformation_f1=misinformation_f1, 
                factual_f1=factual_f1)

def evaluate(split: str) -> dict:
    '''Evaluate the graph neural network.

    Args:
        split (str):
            The split to evaluate the GNN on. Can be 'val' or 'test'.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Get the correct mask, depending on the value of `split`
    mask = val_mask if split == 'val' else test_mask

    gnn.eval()
    with torch.no_grad():

        # Get the input features and the output labels
        input_feats = dict(
            p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
            p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
        )
        output_labels = subgraph.nodes['tweet'].data['label'].cuda()

        # Forward propagation
        logits = gnn(subgraph, input_feats).squeeze()

        # Compute validation loss
        val_loss = F.binary_cross_entropy_with_logits(
            input=logits[mask],
            target=output_labels.float()[mask]
        ).cpu().item()

        # Compute validation metrics
        scores = scorer(logits[mask].ge(0), output_labels[mask])
        val_misinformation_f1 = scores[0].cpu().item()
        val_factual_f1 = scores[1].cpu().item()

    return dict(loss=val_loss, 
                misinformation_f1=val_misinformation_f1, 
                factual_f1=val_factual_f1)

# Initialise optimiser
opt = optim.AdamW(gnn.parameters(), lr=3e-4)

# Initialise scorer
scorer = tm.classification.f_beta.F1Score(num_classes=2, average='none').cuda()

# Initialise dictionary containing validation scores
val_scores = defaultdict(list)

# Initialise progress bar
epoch_pbar = tqdm(range(1000), desc='Training')

for epoch in epoch_pbar:

    # Reset the gradients
    opt.zero_grad()

    # Forward propagation
    train_results = forward_pass()

    # Backward propagation
    train_results['loss'].backward()

    # Update gradients
    opt.step()

    # Evaluate the model
    val_results = evaluate('val')

    # Store the validation scores
    for metric in ['loss', 'misinformation_f1', 'factual_f1']:
        val_scores[metric].append(val_results[metric])

    # Update progress bar description
    if epoch % 25 == 0 and epoch > 0:
        val_loss = np.mean(val_scores['loss'])
        val_misinformation_f1 = np.mean(val_scores['misinformation_f1'])
        val_factual_f1 = np.mean(val_scores['factual_f1'])
        desc = (f'Training - '
                f'loss {train_results["loss"]:.3f} - '
                f'factual_f1 {train_results["factual_f1"]:.3f} - '
                f'misinfo_f1 {train_results["misinformation_f1"]:.3f} - '
                f'val_loss {val_loss:.3f} - '
                f'val_factual_f1 {val_factual_f1:.3f} - '
                f'val_misinfo_f1 {val_misinformation_f1:.3f}')
        epoch_pbar.set_description(desc)
        val_scores = defaultdict(list)

test_results = evaluate('test')
macro_f1 = np.mean([test_results['factual_f1'],
                    test_results['misinformation_f1']])
print()
print('*** Test results ***')
print(f'Factual F1: {test_results["factual_f1"]:.4f}')
print(f'Misinformation F1: {test_results["misinformation_f1"]:.4f}')
print(f'Macro-average F1: {macro_f1:.4f}')

Training - loss 0.172 - factual_f1 0.567 - misinfo_f1 0.965 - val_loss 2.828 - val_factual_f1 0.074 - val_misinfo_f1 0.958: 100%|██████████| 1000/1000 [03:32<00:00,  4.70it/s]


*** Test results ***
Factual F1: 0.1481
Misinformation F1: 0.9469
Macro-average F1: 0.5475


## performance

In [ ]:
model_performance = dict()
model_performance['user_posted_tweet'] = dict(zip(('Factual F1', 'Misinformation F1', 'Macro-average F1', 'Training Time'), (0.1633, 0.9529, 0.5581, 53)))
model_performance['reply_reply_to_tweet'] = dict(zip(('Factual F1', 'Misinformation F1', 'Macro-average F1', 'Training Time'), (0.1644, 0.9280, 0.5462, 6923)))
model_performance['claim_discusses_inv_tweet'] = dict(zip(('Factual F1', 'Misinformation F1', 'Macro-average F1', 'Training Time'), (0.1044, 0.4137, 0.2590, 92)))
model_performance['reply_quote_of_tweet'] = dict(zip(('Factual F1', 'Misinformation F1', 'Macro-average F1', 'Training Time'), (0.1791, 0.9355, 0.5573, 7804)))
model_performance['article_has_article_inv_tweet'] = dict(zip(('Factual F1', 'Misinformation F1', 'Macro-average F1', 'Training Time'), (0.1795, 0.9240, 0.5517, 86)))
model_performance['hashtag_has_hashtag_inv_tweet'] = dict(zip(('Factual F1', 'Misinformation F1', 'Macro-average F1', 'Training Time'), (0.0800, 0.9743, 0.5272, 60)))
model_performance['image_has_image_inv_tweet'] = dict(zip(('Factual F1', 'Misinformation F1', 'Macro-average F1', 'Training Time'), (0.1653, 0.8736, 0.5194, 37)))
model_performance['user_mentions_inv_tweet'] = dict(zip(('Factual F1', 'Misinformation F1', 'Macro-average F1', 'Training Time'), (0.0000, 0.9486, 0.4743, 75)))
model_performance['user_retweeted_tweet'] = dict(zip(('Factual F1', 'Misinformation F1', 'Macro-average F1', 'Training Time'), (0.1455, 0.9457, 0.5456, 233)))


In [ ]:
import pandas as pd
performance = pd.DataFrame(model_performance).T
performance.sort_values(by=['Macro-average F1'], ascending=False)

,Factual F1,Misinformation F1,Macro-average F1,Training Time
user_posted_tweet,0.1633,0.9529,0.5581,53.0
reply_quote_of_tweet,0.1791,0.9355,0.5573,7804.0
article_has_article_inv_tweet,0.1795,0.9240,0.5517,86.0
reply_reply_to_tweet,0.1644,0.9280,0.5462,6923.0
user_retweeted_tweet,0.1455,0.9457,0.5456,233.0
hashtag_has_hashtag_inv_tweet,0.0800,0.9743,0.5272,60.0
image_has_image_inv_tweet,0.1653,0.8736,0.5194,37.0
user_mentions_inv_tweet,0.0000,0.9486,0.4743,75.0
claim_discusses_inv_tweet,0.1044,0.4137,0.2590,92.0
